#  Analysing the solar wind

This is the code from my research project as part of my BSc in Physics. The project is about how the solar wind varies radially outwards from the Sun. 

## Problem
How does the solar wind's velocity, density, temperature and magnetic field vary radially outwards from the Sun? 

## Data Collection
I have obtained data from 6 satellites from a web server via the speasy module. The data is from solar minima and is stored in pandas dataframes. 

## Data Cleaning
Data cleaning and data transformation are performed on the data from each satellite separately. Missing values are interpolated and for some of the satellites the units needed to be converted. The magnetic field coordinates needed to be transformed. The data was then combined.

## Data analysis
A k-nearest neighbours (kNN) regression was applied to the data for each parameter. This is a machine learning algorithm which averages the k closest distance values for each parameter value. 

Speasy code to obtain and plot data from the last solar minimum from ACE

## Importing modules and defining amda_tree

In [ ]:
#Import necessary modules
import time
import datetime as dt

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.style as mplstyle
mplstyle.use('fast')
import numpy as np
import pandas as pd
import speasy as spz
import sklearn
with sklearn.config_context(assume_finite=True):
    pass
import sklearn.metrics as metrics
from sklearn.model_selection import train_test_split
from sklearn import neighbors
from scipy.integrate import solve_ivp
from scipy.spatial.transform import Rotation
import scipy.interpolate as interpolate
import astropy.constants as ac

In [ ]:
!jupyter --version

In [ ]:
#Get amda_tree and sscweb_tree
amda_tree = spz.inventories.tree.amda
sscweb_tree = spz.inventories.tree.amda

Theoretical functions

In [ ]:
def theoretical_density(r):
    return 1.305 * r** (-2.11) * (0.0038 * 1.5 + 4.5)

In [ ]:
def theoretical_velocity(r):
    v_slow = 1.0101 * (3.63 * r ** 0.099)
    v_fast = 1.023 * (483 * r ** 0.099)
    return (v_slow + v_fast)/2

In [ ]:
def theoretical_temperature(r):
    return 1.654*r**(-1.10)*(197*1.5+57300)

# Get Satellite Data

## Coordinate transformations

In [ ]:
# Formula from (Hapgood M, 1992)
MJD = 2400000.5
T0 = (MJD - 51544.5)/36525
M = 357.528 + 35999.05 * T0 + 0.04107 # Sun mean anomaly
sun_mean_longitude = 280.46+36000.772*T0 +0.04107
sun_ecliptic_longitude = sun_mean_longitude+(1.915-0.0048*T0)*np.sin(M)+0.02*np.sin(2*M)

### GSE to HAE Transformation

In [ ]:
# Rotation matrix for transforming GSE to HEE coordinates.
r_hee = Rotation.from_matrix([
    [-1, 0, 0],
    [0, -1, 0],
    [0, 0, 1]])

In [ ]:
# Rotation matrix for transforming HEE to HAE coordinates.
r_hae_from_hee = Rotation.from_matrix([
    [np.cos(-sun_ecliptic_longitude-np.pi), -np.sin(-sun_ecliptic_longitude-np.pi), 0],
    [np.sin(-sun_ecliptic_longitude-np.pi), np.cos(-sun_ecliptic_longitude-np.pi), 0],
    [0, 0, 1]])

In [ ]:
# Transforms GSE coordinates into HAE coordinates
def gse_coordinate_conversion(gse_comp):
    gse = list(gse_comp.values)
    hae_list = []
    r_list = []
    theta_list = []
    phi_list = []
    # The magnetic field components are converted into HAE coordinates.
    for row in zip(gse):
        hee = r_hee.apply(row)
        hae = r_hae_from_hee.apply(hee)
        hae_list.append(hae)
    hae_list = [hae for hae_sublist in hae_list for hae in hae_sublist]
    return hae_list

### RTN to HAE Transformation

In [ ]:
start_date_ = dt.datetime(1996, 2, 1, 0, 0, 0)
end_date_1 = dt.datetime(1997, 3, 1, 0, 0, 0)
ulysses_hae_1996 = spz.amda.get_data("xyz_ulys_hae", start_date_1, end_date_1).to_dataframe()
start_date_2 = dt.datetime(2007, 3, 1, 0, 0, 0)
end_date_2 = dt.datetime(2008, 3, 1, 0, 0, 0)
ulysses_hae_2007 = spz.amda.get_data("xyz_ulys_hae", start_date_2, end_date_2).to_dataframe()
start_date_3 = dt.datetime(2008, 3, 1, 0, 0, 0)
end_date_3 = dt.datetime(2009, 3, 1, 0, 0, 0)
ulysses_hae_2008 = spz.amda.get_data("xyz_ulys_hae", start_date_3, end_date_3).to_dataframe()
start_date_4 = dt.datetime(2009, 3, 1, 0, 0, 0)
end_date_4 = dt.datetime(2009, 9, 1, 0, 0, 0)
ulysses_hae_2009 = spz.amda.get_data("xyz_ulys_hae", start_date_4, end_date_4).to_dataframe()

In [ ]:
ulysses_hae = pd.concat([ulysses_hae_1996, ulysses_hae_2007, ulysses_hae_2008, ulysses_hae_2009])
ulysses_hae_comp = ulysses_hae[["xyz_ulys_hae[0]", "xyz_ulys_hae[1]", "xyz_ulys_hae[2]"]]

In [ ]:
# Rotation matrix for transformation from HAE to HCD coordinates.
omega = 75.6*np.pi/180+1.367*np.pi/180*T0
r_hcd_from_hae = Rotation.from_matrix(np.matmul(np.matmul([
    [np.cos(omega), -np.sin(omega), 0],
    [np.sin(omega), np.cos(omega), 0],
    [0, 0, 1]], [
    [1, 0, 0],
    [0, np.cos(75.6*np.pi/180), -np.sin(75.6*np.pi/180)],
    [0, np.sin(75.6*np.pi/180), np.cos(75.6*np.pi/180)]]), [
    [np.cos(0), -np.sin(0), 0],
    [np.sin(0), np.cos(0), 0],
    [0, 0, 1]]))

In [ ]:
def hae_hcd_coordinate_conversion(hae_comp):
    hae = list(hae_comp.values)
    hcd_list = []
    r_list = []
    theta_list = []
    phi_list = []
    for row in zip(hae):
        hcd = r_hcd_from_hae.apply(row)
        hcd_list.append(hcd)
    return hcd_list

In [ ]:
# Rotation matrix for transforming HGRTN to HCD coordinates.
# E = <psi, Z>*<theta,X>*<omega,Z>
# E(-psi+90, -theta, -90)
r_hcd = Rotation.from_matrix([
    np.matmul(np.matmul([
        [np.cos(-psi-np.pi/2), -np.sin(-psi-np.pi/2), 0],
        [np.sin(-psi-np.pi/2), np.cos(-psi-np.pi/2), 0],
        [0, 0, 1]], [
        [1, 0, 0],
        [0, np.cos(-theta), -np.sin(-theta)],
        [0, np.sin(-theta), np.cos(-theta)]]), [
        [np.cos(-np.pi/2), -np.sin(-np.pi/2), 0],
        [np.sin(-np.pi/2), np.cos(-np.pi/2), 0],
        [0, 0, 1]])])

In [ ]:
# Rotation matrix for transformation from HCD to HAE coordinates.
omega = 75.6*np.pi/180+1.367*np.pi/180*T0
r_hae_from_hcd = Rotation.from_matrix(np.matmul(np.matmul([
    [np.cos(-omega), -np.sin(-omega), 0],
    [np.sin(-omega), np.cos(-omega), 0],
    [0, 0, 1]], [
    [1, 0, 0],
    [0, np.cos(-75.6*np.pi/180), -np.sin(-75.6*np.pi/180)],
    [0, np.sin(-75.6*np.pi/180), np.cos(-75.6*np.pi/180)]]), [
    [np.cos(0), -np.sin(0), 0],
    [np.sin(0), np.cos(0), 0)],
    [0, 0, 1]]))

In [ ]:
# Transform HCRTN into heliocentric polar coordinates.
def rtn_coordinate_conversion(rtn_comp):
    rtn = list(rtn_comp.values)
    hae_list = []
    r_list = []
    theta_list = []
    phi_list = []
    for row in zip(rtn):
        hcd = r_hcd.apply(rtn)
        hae = r_hae_from_hcd.apply(hcd)
        hae_list.append(hae)
    return hae_list

In [ ]:
# The magnetic field HAE coordinates are converted into spherical polar coordinates.
def polar_coordinate_conversion(comp, coords, hae_list):
    if coords == "gse":
        gse_coordinate_conversion(comp)
    if coords == "rtn":
        rtn_coordinate_conversion(comp)
    for i in hae_list:
        h = [x for xs in i for x in xs]
        r = np.sqrt(h[0]**2+h[1]**2+h[2]**2)
        r_list.append(r)
        theta_list.append(np.arccos(h[2]/(r)))
        phi_list.append(np.sign(h[1])*np.arccos(h[0]/(np.sqrt(h[0]**2+h[1]**2))))
    return [r_list, theta_list, phi_list]

## ACE

Attempting to get the magntiude of the magnetic field from ACE using AMDA

In [ ]:
start_date = dt.datetime(2018, 6, 1, 0, 0)
end_date = dt.datetime(2020, 7, 1, 0, 0)

In [ ]:
# Get data for the whole period during the solar minimum.
ace_dist_sun = spz.get_data(amda_tree.Parameters.ACE.Ephemeris.ace_orb_all.ace_r_sun, start_date, end_date)
ace_dist_sun = ace_dist_sun.to_dataframe()
ace_density = spz.get_data(amda_tree.Parameters.ACE.SWEPAM.ace_swe_all.sw_n, start_date, end_date)
ace_density = ace_density.to_dataframe()
ace_velocity = spz.amda.get_data("sw_v_gsm", start_date, end_date)
ace_velocity = ace_velocity.to_dataframe()
ace_temperature = spz.get_data(amda_tree.Parameters.ACE.SWEPAM.ace_swe_all.sw_t, start_date, end_date)
ace_temperature = ace_temperature.to_dataframe()
ace_mag_field = spz.get_data(amda_tree.Parameters.ACE.MFI.ace_imf_all.imf, start_date, end_date)
ace_mag_field = ace_mag_field.to_dataframe()

In [ ]:
ace_dist_sun.to_csv("ace_distance.csv")
ace_density.to_csv("ace_density.csv")
ace_velocity.to_csv("ace_velocity.csv")
ace_temperature.to_csv("ace_temperature.csv")
ace_mag_field.to_csv("ace_mag_field")

In [ ]:
ace_dist_sun = pd.read_csv("ace_distance.csv")
ace_density = pd.read_csv("ace_density.csv")
ace_velocity = pd.read_csv("ace_velocity.csv")
ace_temperature = pd.read_csv("ace_temperature.csv")
ace_mag_field = pd.read_csv("ace_mag_field")

In [ ]:
ace_dist_sun.reset_index(inplace=True)
ace_density.reset_index(inplace=True)
ace_temperature.reset_index(inplace=True)
ace_mag_field.reset_index(inplace=True)
ace_dist_sun = ace_dist_sun.rename(columns = {"Unnamed: 0": "Time"})
ace_density = ace_density.rename(columns = {"Unnamed: 0": "Time"})
ace_velocity = ace_velocity.rename(columns = {"Unnamed: 0": "Time", "sw_v_gsm[0]": "vx", "sw_v_gsm[1]": "vy", "sw_v_gsm[2]": "vz"})
ace_temperature = ace_temperature.rename(columns = {"Unnamed: 0": "Time"})
ace_mag_field = ace_mag_field.rename(columns = {"Unnamed: 0": "Time"})

In [ ]:
print(ace_dist_sun.head())
print(ace_density.tail())

In [ ]:
for start_time, end_time in zip(icme_starts[:-1], icme_ends):
    ace_dist_sun[dt.datetime(ace_dist_sun["Time"].to_numpy()) < start_time | (dt.datetime(ace_dist_sun["Time"]) > end_time)]
    ace_density[dt.datetime(ace_density["Time"]) < start_time | (dt.datetime(ace_density["Time"]) > end_time)]
    ace_velocity[dt.datetime(ace_velocity["Time"]) < start_time | (dt.datetime(ace_velocity["Time"]) > end_time)]
    ace_temperature[dt.datetime(ace_temperature["Time"]) < start_time | (dt.datetime(ace_temperature["Time"]) > end_time)]
    ace_mag_field[dt.datetime(ace_mag_field["Time"]) < start_time | (dt.datetime(ace_mag_field["Time"]) > end_time)]

In [ ]:
#Manipulate the dataframes to get them into a useable form
ace_dist_sun["Time"] = (pd.to_datetime(ace_dist_sun["Time"])-start_date).dt.total_seconds()
ace_density["Time"] = (pd.to_datetime(ace_density["Time"])-start_date).dt.total_seconds()
ace_velocity["Time"] = (pd.to_datetime(ace_velocity["Time"])-start_date).dt.total_seconds()
ace_temperature["Time"] = (pd.to_datetime(ace_temperature["Time"])-start_date).dt.total_seconds()
ace_mag_field["Time"] = (pd.to_datetime(ace_mag_field["Time"])-start_date).dt.total_seconds()
ace_velocity = ace_velocity.rename(columns = {"index": "Time", "sw_v_gsm[0]": "vx", "sw_v_gsm[1]": "vy", "sw_v_gsm[2]": "vz"})
ace_velocity = ace_velocity.assign(vmag = lambda x: np.sqrt(x["vx"]**2 + x["vy"]**2 + x["vz"]**2))
ace_temperature = ace_temperature.assign(temp = lambda x: (x["sw_t"]*1.6*10**(-19))/(1.38*10**(-23)))
print(ace_dist_sun.head())
print(ace_density.tail())

In [ ]:
#Array of desired values
point = 0
interpolated_times = []
while point <= 65749680:
    interpolated_times.append(point)
    point += 63.99923

In [ ]:
#Interpolation
interpolate_function = interpolate.interp1d(ace_dist_sun["Time"], ace_dist_sun["ace_r_sun"])
interpolated_distances = interpolate_function(interpolated_times) 

In [ ]:
#Plotting r against n
plt.plot(interpolated_distances, ace_density["sw_n"])
plt.plot(interpolated_distances, theoretical_density(interpolated_distances))
plt.xlabel("Distance from the Sun (AU)")
plt.ylabel("Number density (cm^-3)")
plt.show()

In [ ]:
#Plotting v against r
plt.plot(interpolated_distances, ace_velocity["vmag"])
plt.plot(interpolated_distances, theoretical_velocity(interpolated_distances))
plt.xlabel("Distance from the Sun (AU)")
plt.ylabel("Magnitude of the velocity (km/s)")
plt.legend(["Satellite Data", "Theoretical Prediction"])
plt.show()

In [ ]:
#Plotting T against r
plt.plot(interpolated_distances, ace_temperature["temp"])
plt.plot(interpolated_distances, theoretical_temperature(interpolated_distances))
plt.xlabel("Distance from the Sun (AU)")
plt.ylabel("Temperature (K)")
plt.show()

In [ ]:
ace_mag_field.head()

In [ ]:
# Convert the magnetic field coordinates into heliocentric polar coordinates.
ace_gse_comp = ace_mag_field[["imf[0]", "imf[1]", "imf[2]"]]
ace_polar_coords = gse_coordinate_conversion(ace_gse_comp)
ace_polar_coords = pd.DataFrame(ace_polar_coords, columns=["Br", "B_theta", "B_phi"])
print(ace_polar_coords.head())

In [ ]:
ace_mag_field = ace_mag_field.assign(Br=ace_polar_coords["Br"])
ace_mag_field = ace_mag_field.assign(B_theta=ace_polar_coords["B_theta"])
ace_mag_field = ace_mag_field.assign(B_phi=ace_polar_coords["B_phi"])
print(ace_mag_field.tail())

In [ ]:
print(gse_coordinate_conversion(ace_gse_comp))

In [ ]:
#Plotting T against r
plt.plot(interpolated_distances, np.average(ace_mag_field["Br"].dropna().to_numpy().reshape(-1, 4)))
plt.xlabel("Distance from the Sun (AU)")
plt.ylabel("Br (nT)")
plt.show()

In [ ]:
ace_distances = {"distance": interpolated_distances}
ace_data = pd.DataFrame(ace_distances)
ace_data = ace_data.assign(density = ace_density["sw_n"])
ace_data = ace_data.assign(vmag = ace_velocity["vmag"])
ace_data = ace_data.assign(temp = ace_temperature["temp"])
ace_data = ace_data.assign(Br = ace_mag_field["Br"])
ace_data = ace_data.assign(B_theta = ace_mag_field["B_theta"])
ace_data = ace_data.assign(B_phi = ace_mag_field["B_phi"])
ace_data = ace_data.to_csv("ace_data.csv")
#print(ace_data.head())

## DSCOVR

In [ ]:
#Set datetime range as variables for ease of use
start_date = dt.datetime(2018, 6, 1, 0, 0)
end_date = dt.datetime(2020, 7, 1, 0, 0)

In [ ]:
# Obtain the distance and density data from DSCOVR and convert into a dataframe 
dscovr_dist_sun = spz.get_data(amda_tree.Parameters.DSCOVR.Ephemeris.dsc_orb_all.dsc_r_sun, start_date, end_date)
dscovr_dist_sun = dscovr_dist_sun.to_dataframe()
dscovr_density = spz.amda.get_data("dsc_npr_3s", start_date, end_date)
dscovr_density = dscovr_density.to_dataframe()
dscovr_velocity = spz.amda.get_data("dsc_vpr_3s_gsm", start_date, end_date)
dscovr_velocity = dscovr_velocity.to_dataframe()
dscovr_temperature = spz.amda.get_data("dsc_tpr_3s", start_date, end_date)
dscovr_temperature = dscovr_temperature.to_dataframe()
dscovr_mag_field = spz.amda.get_data("dsc_b_gse", start_date, end_date)
dscovr_mag_field = dscovr_mag_field.to_dataframe()
print(dscovr_dist_sun.head())

In [ ]:
dscovr_dist_sun.to_csv("dscovr_distance.csv")
dscovr_density.to_csv("dscovr_density.csv")
dscovr_velocity.to_csv("dscovr_velocity.csv")
dscovr_temperature.to_csv("dscovr_temperature.csv")
dscovr_mag_field.to_csv("dscovr_mag_field")

In [ ]:
dscovr_dist_sun = pd.read_csv("dscovr_distance.csv")
dscovr_density = pd.read_csv("dscovr_density.csv")
dscovr_velocity = pd.read_csv("dscovr_velocity.csv")
dscovr_temperature = pd.read_csv("dscovr_temperature.csv")
dscovr_mag_field = pd.read_csv("dscovr_mag_field")

In [ ]:
print(dscovr_velocity.head())
print(dscovr_temperature.head())

In [ ]:
# Manipulate the dataframes to get them into a useable form
dscovr_dist_sun.reset_index(inplace=True)
dscovr_density.reset_index(inplace=True)
dscovr_temperature.reset_index(inplace=True)
dscovr_mag_field.reset_index(inplace=True)
dscovr_dist_sun = dscovr_dist_sun.rename(columns = {"Unnamed: 0": "Time"})
dscovr_density = dscovr_density.rename(columns = {"Unnamed: 0": "Time"})
dscovr_velocity = dscovr_velocity.rename(columns = {"Unnamed: 0": "Time"})
dscovr_temperature = dscovr_temperature.rename(columns = {"Unnamed: 0": "Time"})
dscovr_mag_field = dscovr_mag_field.rename(columns = {"Unnamed: 0": "Time"})
dscovr_dist_sun["Time"] = (pd.to_datetime(dscovr_dist_sun["Time"])-start_date).dt.total_seconds()
dscovr_density["Time"] = (pd.to_datetime(dscovr_density["Time"])-start_date).dt.total_seconds()
dscovr_velocity["Time"] = (pd.to_datetime(dscovr_velocity["Time"])-start_date).dt.total_seconds()
dscovr_temperature["Time"] = (pd.to_datetime(dscovr_temperature["Time"])-start_date).dt.total_seconds()
dscovr_mag_field["Time"] = (pd.to_datetime(dscovr_mag_field["Time"])-start_date).dt.total_seconds()
dscovr_velocity = dscovr_velocity.rename(columns = {"index": "Time", "dsc_vpr_3s_gsm[0]": "vx", "dsc_vpr_3s_gsm[1]": "vy", "dsc_vpr_3s_gsm[2]": "vz"})
dscovr_velocity = dscovr_velocity.assign(vmag = lambda x: np.sqrt(x["vx"]**2 + x["vy"]**2 + x["vz"]**2))
# DSCOVR's temperature is in eV to convert into K
dscovr_temperature = dscovr_temperature.assign(temp = lambda x: np.absolute((x["dsc_tpr_3s"]*(1.6*10**(-19)))/(1.38*10**(-23))))
print(dscovr_dist_sun.head())
print(dscovr_density.head())
print(dscovr_temperature.head())

In [ ]:
print(dscovr_dist_sun.tail())
print(dscovr_density.tail())

In [ ]:
#Array of desired values
point = 27
interpolated_times = []
while point <= 65404740:
    interpolated_times.append(point)
    point += 27.8306

In [ ]:
interpolate_func_d = interpolate.interp1d(dscovr_dist_sun["Time"], dscovr_dist_sun["dsc_r_sun"])
interpolated_distances = interpolate_func_d(interpolated_times)
interpolate_func_bx = interpolate.interp1d(dscovr_mag_field["Time"], dscovr_mag_field["dsc_b_gse[0]"])
interpolated_bx = interpolate_func_bx(interpolated_times)
interpolate_func_by = interpolate.interp1d(dscovr_mag_field["Time"], dscovr_mag_field["dsc_b_gse[1]"])
interpolated_by = interpolate_func_by(interpolated_times)
interpolate_func_bz = interpolate.interp1d(dscovr_mag_field["Time"], dscovr_mag_field["dsc_b_gse[2]"])
interpolated_bz = interpolate_func_bz(interpolated_times)

In [ ]:
interpolated_mag = pd.DataFrame({"Time": interpolated_times, "Bx" : interpolated_bx, "By" : interpolated_by, "Bz" : interpolated_bz})

In [ ]:
interpolated_mag["Bx"]

In [ ]:
# Convert the magnetic field coordinates into heliocentric polar coordinates.
dscovr_gse_comp = interpolated_mag[["Bx", "By", "Bz"]]
dscovr_polar_coords = gse_coordinate_conversion(dscovr_gse_comp)
dscovr_polar_coords = pd.DataFrame(dscovr_polar_coords, columns=["Br", "B_theta", "B_phi"])
print(dscovr_polar_coords.head())

In [ ]:
print(len(interpolated_distances))
print(len(interpolated_bx))

In [ ]:
#Plotting r against n
plt.plot(interpolated_distances, dscovr_density["dsc_npr_3s"])
plt.plot(interpolated_distances, theoretical_density(interpolated_distances))
plt.xlabel("Distance from the Sun (AU)")
plt.ylabel("Number density (cm^-3)")
plt.show()

In [ ]:
#Plotting r against n
plt.plot(interpolated_distances, dscovr_velocity["vmag"])
plt.plot(interpolated_distances, theoretical_velocity(interpolated_distances))
plt.xlabel("Distance from the Sun (AU)")
plt.ylabel("Magntiude of the velocity (km/s)")
plt.show()

In [ ]:
plt.plot(interpolated_distances, dscovr_temperature["temp"])
plt.plot(interpolated_distances, theoretical_temperature(interpolated_distances))
plt.xlabel("Distance from the Sun (AU)")
plt.ylabel("Temperature(K)")
plt.show()

In [ ]:
plt.plot(interpolated_distances, dscovr_polar_coords["Br"])
plt.xlabel("Distance from the Sun (AU)")
plt.ylabel("Br (nT)")
plt.show()

In [ ]:
dscovr_distances = {"distance": interpolated_distances}
dscovr_data = pd.DataFrame(dscovr_distances)
dscovr_data = dscovr_data.assign(density=dscovr_density["dsc_npr_3s"])
dscovr_data = dscovr_data.assign(vmag=dscovr_velocity["vmag"])
dscovr_data = dscovr_data.assign(temp=dscovr_temperature["temp"])
dscovr_data = dscovr_data.assign(Br=dscovr_polar_coords["Br"])
dscovr_data = dscovr_data.assign(B_theta=dscovr_polar_coords["B_theta"])
dscovr_data = dscovr_data.assign(B_phi=dscovr_polar_coords["B_phi"])
dscovr_data = dscovr_data.to_csv("dscovr_data.csv")

## Helios 1

In [ ]:
#Set datetime range as variables for ease of use
start_date = dt.datetime(1975, 4, 1, 0, 0)
end_date = dt.datetime(1977, 5, 1, 0, 0)

### Density against distance from the Sun

In [ ]:
#Obtain the distance and density data from Helios 1 and convert into a dataframe 
helios1_dist_sun = spz.amda.get_data("helios1_sun_r", start_date, end_date)
helios1_dist_sun = helios1_dist_sun.to_dataframe()
#Obtain the density data
helios1_density = spz.amda.get_data("helios1_e1_np", start_date, end_date)
helios1_density = helios1_density.to_dataframe()
helios1_velocity = spz.amda.get_data("helios1_e1_vp", start_date, end_date)
helios1_velocity = helios1_velocity.to_dataframe()
helios1_temperature = spz.amda.get_data("helios1_e1_t_perp", start_date, end_date)
helios1_temperature = helios1_temperature.to_dataframe()
helios1_mag_field = spz.amda.get_data("helios1_b_rtn",start_date, end_date)
helios1_mag_field = helios1_mag_field.to_dataframe()
#print(helios1_dist_sun.head())

In [ ]:
#Manipulate the dataframes to get them into a useable form
helios1_dist_sun.reset_index(inplace=True)
helios1_density.reset_index(inplace=True)
helios1_velocity.reset_index(inplace=True)
helios1_temperature.reset_index(inplace=True)
helios1_mag_field.reset_index(inplace=True)
helios1_dist_sun = helios1_dist_sun.rename(columns ={"index": "Time"})
helios1_density = helios1_density.rename(columns = {"index": "Time"})
helios1_velocity = helios1_velocity.rename(columns = {"index": "Time", "helios1_e1_vp[0]": "vx", "helios1_e1_vp[1]": "vy", "helios1_e1_vp[2]": "vz"})
helios1_temperature = helios1_temperature.rename(columns = {"index": "Time"})
helios1_mag_field = helios1_mag_field.rename(columns = {"index": "Time"})
helios1_dist_sun["Time"] = (pd.to_datetime(helios1_dist_sun["Time"])-start_date).dt.total_seconds()
helios1_density["Time"] = (pd.to_datetime(helios1_density["Time"])-start_date).dt.total_seconds()
helios1_velocity["Time"] = (pd.to_datetime(helios1_velocity["Time"])-start_date).dt.total_seconds()
helios1_temperature["Time"] = (pd.to_datetime(helios1_temperature["Time"])-start_date).dt.total_seconds()
helios1_mag_field["Time"] = (pd.to_datetime(helios1_mag_field["Time"])-start_date).dt.total_seconds()
helios1_velocity = helios1_velocity.assign(vmag = lambda x: np.sqrt(x["vr"]**2 + x["vt"]**2 + x["vn"]**2))
helios1_temperature = helios1_temperature.assign(temp = lambda x: x["t_perp"]*1.6*10**(-19)/(1.38*10**(-23)))

In [ ]:
helios1_dist_sun = helios1_dist_sun.to_csv("helios1_raw_dist.csv")
helios1_density = helios1_density.to_csv("helios1_raw_den")
helios1_velocity = helios1_velocity.to_csv("helios1_raw_vmag")
helios1_temperature = helios1_temperature.to_csv("helios1_raw_temp.csv")
helios1_mag_field = helios1_mag_field.to_csv("helios1_raw_mag.csv")

In [ ]:
fig = plt.figure()
plt.plot(helios1_density["Time"], helios1_density["density"])
plt.show()

In [ ]:
print(helios1_dist_sun.head())
print(helios1_velocity.tail())
print(helios1_mag_field.tail())

In [ ]:
#Array of desired values
point = 3599
interpolated_times = []
while point <= 65750400:
    interpolated_times.append(point)
    point += 181.6392

In [ ]:
#Interpolation
interpolate_function = interpolate.interp1d(helios1_dist_sun["Time"], helios1_dist_sun["distance helios1-sun"])
interpolated_distances = interpolate_function(interpolated_times)

In [ ]:
#Plotting n against r
fig = plt.figure()
plt.plot(interpolated_distances, helios1_density["density"])
plt.plot(interpolated_distances, theoretical_density(interpolated_distances))
plt.xlabel("Distance from the Sun (AU)")
plt.ylabel("Number density(cm/3)")
plt.show()
print(helios1_density.tail())

In [ ]:
fig = plt.figure()
plt.plot(helios1_velocity["Time"], helios1_velocity["vmag"])
plt.show()
print(helios1_dist_sun.tail())
print(helios1_velocity.tail())

In [ ]:
fig = plt.figure()
plt.plot(interpolated_distances, helios1_velocity["vmag"])
plt.plot(interpolated_distances, theoretical_velocity(interpolated_distances))
plt.xlabel("Distance from the Sun (AU)")
plt.ylabel("Magnitude of the velocity (km/s)")
plt.show()

In [ ]:
plt.plot(interpolated_distances, helios1_temperature["temp"])
plt.plot(interpolated_distances, theoretical_temperature(interpolated_distances))
plt.xlabel("Distance from the Sun (AU)")
plt.ylabel("Temperature (K)")
plt.show()

In [ ]:
plt.plot(interpolated_distances, helios1_mag_field["r"])
plt.xlabel("Distance from the Sun (AU)")
plt.ylabel("Bx (nT)")
plt.show()

In [ ]:
helios1_distances = {"distance": interpolated_distances}
helios1_data = pd.DataFrame(helios1_distances)
helios1_data = helios1_data.assign(density = helios1_density["density"])
helios1_data = helios1_data.assign(vmag = helios1_velocity["vmag"])
helios1_data = helios1_data.assign(temp = helios1_temperature["temp"])
helios1_data = helios1_data.assign(Bx = helios1_mag_field["r"])
helios1_data = helios1_data.assign(By = helios1_mag_field["t"])
helios1_data = helios1_data.assign(Bz = helios1_mag_field["n"])
helios1_data = helios1_data.to_csv("helios1_data.csv")

## Parker Solar Probe

In [ ]:
#Set datetime range as variables for ease of use
start_date = dt.datetime(2018, 6, 1, 0, 0)
end_date = dt.datetime(2020, 7, 1, 0, 0)

### Density against distance from the Sun

In [ ]:
#Obtain the distance and density data from ACE and convert into a dataframe 
psp_dist_sun = spz.get_data(amda_tree.Parameters.PSP.Ephemeris.psp_orb_all.psp_r_sun, start_date, end_date)
psp_dist_sun = psp_dist_sun.to_dataframe()
#Obtain the density data
psp_density = spz.amda.get_data("psp_spc_np_mom", start_date, end_date)
psp_density = psp_density.to_dataframe()
psp_velocity = spz.amda.get_data("psp_spc_vp_mom", start_date, end_date)
psp_velocity = psp_velocity.to_dataframe()
psp_temperature = spz.amda.get_data("psp_spi_Hw", start_date, end_date)
psp_temperature = psp_temperature.to_dataframe()
psp_mag_field = spz.amda.get_data("psp_b_1min", start_date, end_date)
psp_mag_field = psp_mag_field.to_dataframe()

In [ ]:
psp_density.head()

In [ ]:
#Manipulate the dataframes to get them into a useable form
psp_dist_sun.reset_index(inplace=True)
psp_density.reset_index(inplace=True)
psp_velocity.reset_index(inplace=True)
psp_temperature.reset_index(inplace=True)
psp_mag_field.reset_index(inplace=True)
psp_dist_sun = psp_dist_sun.rename(columns = {"index": "Time"})
psp_density = psp_density.rename(columns = {"index": "Time"})
psp_velocity = psp_velocity.rename(columns = {"index": "Time"})
psp_temperature = psp_temperature.rename(columns = {"index": "Time"})
psp_mag_field = psp_mag_field.rename(columns = {"index": "Time"})
psp_dist_sun["Time"] = (pd.to_datetime(psp_dist_sun["Time"])-start_date).dt.total_seconds()
psp_density["Time"] = (pd.to_datetime(psp_density["Time"])-start_date).dt.total_seconds()
psp_velocity["Time"] = (pd.to_datetime(psp_velocity["Time"])-start_date).dt.total_seconds()
psp_temperature["Time"] = (pd.to_datetime(psp_temperature["Time"])-start_date).dt.total_seconds()
psp_mag_field["Time"] = (pd.to_datetime(psp_mag_field["Time"])-start_date).dt.total_seconds()
psp_velocity = psp_velocity.assign(vmag = lambda x: np.sqrt(x["vpr"]**2 + x["vpt"]**2 + x["vpn"]**2))
psp_temperature = psp_temperature.assign(temp = lambda x: x["h+ temperature"]*1.6*10**(-19)/(1.38*10**(-23)))
print(psp_dist_sun.head())
print(psp_density.tail())

In [ ]:
fig = plt.figure()
plt.plot(psp_velocity["Time"], psp_velocity["vmag"])
plt.show()

In [ ]:
fig = plt.figure()
plt.plot(psp_density["Time"], psp_density["np_mom"])
plt.show()

In [ ]:
psp_mag_field.tail()

In [ ]:
count = 302400
interpolated_times = []
while count <= 65579440:
    interpolated_times.append(count)
    count = count + 99.66158

In [ ]:
#Interpolation
interpolate_function = interpolate.interp1d(psp_dist_sun["Time"], psp_dist_sun["distance psp-sun"])
interpolated_distances = interpolate_function(interpolated_times)
interpolate_func_den = interpolate.interp1d(psp_density["Time"], psp_density["np_mom"])
interpolated_densities = interpolate_func_den(interpolated_times)
interpolate_func_vel = interpolate.interp1d(psp_velocity["Time"], psp_velocity["vmag"])
interpolated_velocities = interpolate_func_vel(interpolated_times)
interpolate_func_temp = interpolate.interp1d(psp_temperature["Time"], psp_temperature["temp"])
interpolated_temperatures = interpolate_func_temp(interpolated_times)

In [ ]:
fig = plt.figure()
plt.plot(interpolated_distances, interpolated_densities)
plt.plot(interpolated_distances, theoretical_density(interpolated_distances))
plt.xlabel("Distance from the Sun (AU)")
plt.ylabel("Number density(cm/3)")
plt.show()

In [ ]:
fig = plt.figure()
plt.plot(interpolated_distances, interpolated_velocities)
plt.plot(interpolated_distances, theoretical_velocity(interpolated_distances))
plt.xlabel("Distance from the Sun (AU)")
plt.ylabel("Magnitude of the velocity (km/s)")
plt.show()

In [ ]:
psp_temperature.tail()

In [ ]:
plt.plot(interpolated_distances, interpolated_temperatures)
plt.show()

In [ ]:
count = 10641940
interpolated_times_temp = []
while count <= 65579440:
    interpolated_times_temp.append(count)
    count = count + 3.5660694

In [ ]:
psp_temperature.dropna(inplace=True)
interpolate_func_temp = interpolate.interp1d(psp_temperature["Time"], psp_temperature["temp"])
interpolated_temp = interpolate_func_temp(interpolated_times_temp)

In [ ]:
print(interpolated_temp[~(np.isnan(interpolated_temp))])

In [ ]:
plt.plot(interpolated_distances, interpolated_temp)
plt.plot(interpolated_distances, theoretical_temperature(interpolated_distances), color="orange")
plt.xlabel("Distance from the Sun (AU)")
plt.ylabel("Temperature(K)")
plt.show()

In [ ]:
print(psp_dist_sun.tail())
print(psp_mag_field.tail())

In [ ]:
count = 32
interpolated_times = []
while count <= 2505572:
    interpolated_times.append(count)
    count = count + 74.06

In [ ]:
#Interpolation
interpolate_func_d = interpolate.interp1d(psp_dist_sun["Time"], psp_dist_sun["psp_r_sun"])
interpolated_distances = interpolate_func_d(interpolated_times)
interpolate_func_bx = interpolate.interp1d(psp_mag_field["Time"], psp_mag_field["psp_b_1min[0]"].to_numpy().astype(dtype="float64"))
interpolated_bx = interpolate_func_bx(interpolated_times)
interpolate_func_by = interpolate.interp1d(psp_mag_field["Time"], psp_mag_field["psp_b_1min[1]"])
interpolated_by = interpolate_func_by(interpolated_times)
interpolate_func_bz = interpolate.interp1d(psp_mag_field["Time"], psp_mag_field["psp_b_1min[2]"])
interpolated_bz = interpolate_func_bz(interpolated_times)

In [ ]:
plt.plot(interpolated_distances, interpolated_bx)
plt.xlabel("Distance from the Sun (AU)")
plt.ylabel("Bx (nT)")
plt.show()

In [ ]:
psp_distances = {"distance": interpolated_distances}
psp_data = pd.DataFrame(psp_distances)
psp_data = psp_data.assign(density = interpolated_densities)
psp_data = psp_data.assign(vmag = interpolated_velocities)
psp_data = psp_data.assign(temp = interpolated_temperatures)
psp_data = psp_data.assign(Bx = psp_mag_field["br"])
psp_data = psp_data.assign(By = psp_mag_field["bt"])
psp_data = psp_data.assign(Bz = psp_mag_field["bn"])
psp_data = psp_data.to_csv("psp_data.csv")

## Voyager 1

In [ ]:
start_date = dt.datetime(1985, 8, 1, 0, 0, 0)
end_date = dt.datetime(1987, 2, 1, 0, 0, 0)

In [ ]:
voyager1_dist_sun = spz.amda.get_data("r_vo1", start_date, end_date)
voyager1_dist_sun = voyager1_dist_sun.to_dataframe()
voyager1_density = spz.amda.get_data("vo1_dens_full", start_date, end_date)
voyager1_density = voyager1_density.to_dataframe()

In [ ]:

voyager1_velocity = spz.amda.get_data("vo1_vel_full", start_date, end_date)
voyager1_velocity = voyager1_velocity.to_dataframe()
voyager1_temperature = spz.amda.get_data("vo1_temp_full", start_date, end_date)
voyager1_temperature = voyager1_temperature.to_dataframe()
voyager1_mag_field = spz.amda.get_data("vo1_b_full", start_date, end_date)
voyager1_mag_field = voyager1_mag_field.to_dataframe()

In [ ]:
#Manipulate the dataframes to get them into a useable form
voyager1_dist_sun.reset_index(inplace=True)
voyager1_density.reset_index(inplace=True)
voyager1_velocity.reset_index(inplace=True)
voyager1_temperature.reset_index(inplace=True)
voyager1_mag_field.reset_index(inplace=True)
voyager1_dist_sun = voyager1_dist_sun.rename(columns = {"index": "Time"})
voyager1_density = voyager1_density.rename(columns = {"index": "Time"})
voyager1_velocity = voyager1_velocity.rename(columns = {"index": "Time"})
voyager1_temperature = voyager1_temperature.rename(columns = {"index": "Time"})
voyager1_mag_field = voyager1_mag_field.rename(columns = {"index": "Time"})
voyager1_dist_sun["Time"] = (pd.to_datetime(voyager1_dist_sun["Time"])-start_date).dt.total_seconds()
voyager1_density["Time"] = (pd.to_datetime(voyager1_density["Time"])-start_date).dt.total_seconds()
voyager1_velocity["Time"] = (pd.to_datetime(voyager1_velocity["Time"])-start_date).dt.total_seconds()
voyager1_temperature["Time"] = (pd.to_datetime(voyager1_temperature["Time"])-start_date).dt.total_seconds
voyager1_mag_field["Time"] = (pd.to_datetime(voyager1_mag_field["Time"])-start_date).dt.total_seconds()
voyager1_temperature = voyager1_temperature.assign(temp = lambda x: x["vo1_temp_full"]*1.6*10**(-19)/(1.38*10**(-23)))
print(voyager1_dist_sun.head())
print(voyager1_density.head())

In [ ]:
print(voyager1_dist_sun.tail())
print(voyager1_density.tail())
print(voyager1_temperature.head())
count = 21600
interpolated_times = []
while count <= 2649600:
    interpolated_times.append(count)
    count = count + 3600

In [ ]:
#Interpolation
interpolate_function = interpolate.interp1d(voyager1_dist_sun["Time"], voyager1_dist_sun["r_vo1"])
interpolated_distances = interpolate_function(interpolated_times)

In [ ]:
ix = [731, 732, 733, 734, 735, 736, 737, 738, 739, 740, 741, 742]
voyager1_density = voyager1_density.drop(index=ix)
voyager1_velocity = voyager1_velocity.drop(index=ix)
voyager1_temperature = voyager1_temperature.drop(index=ix)

In [ ]:
fig = plt.figure()
plt.plot(interpolated_distances, voyager1_density["vo1_dens_full"])
plt.plot(interpolated_distances, theoretical_density(interpolated_distances))
plt.xlabel("Distance from the Sun (AU)")
plt.ylabel("Number density(cm^-3)")
plt.show()

In [ ]:
print(voyager1_dist_sun.tail())
print(voyager1_velocity.tail())
count = 21600
interpolated_times = []
while count <= 2649600:
    interpolated_times.append(count)
    count = count + 3600

In [ ]:
#Interpolation
interpolate_function = interpolate.interp1d(voyager1_dist_sun["Time"], voyager1_dist_sun["r_vo1"])
interpolated_distances = interpolate_function(interpolated_times)

In [ ]:
fig = plt.figure()
plt.plot(interpolated_distances, voyager1_velocity["vo1_vel_full"])
plt.plot(interpolated_distances, theoretical_velocity(interpolated_distances))
plt.legend(["Satellite Data", "Theoretical Prediction"])
plt.ylabel("Velocity (km/s)")
plt.xlabel("Distance from the Sun (AU)")
plt.show()

In [ ]:
plt.plot(interpolated_distances, voyager1_temperature["temp"])
plt.plot(interpolated_distances, theoretical_temperature(interpolated_distances))
plt.xlabel("Distance from the Sun (AU)")
plt.ylabel("Temperature (K)")
plt.show()

In [ ]:
ix = []
count =731
while count <=743:
    ix.append(count)
    count = count + 1
voyager1_mag_field.drop(index=ix, inplace=True)

In [ ]:
plt.plot(interpolated_distances, voyager1_mag_field["vo1_b_full[0]"])
plt.xlabel("Distance from the Sun (AU)")
plt.ylabel("Bx (nT)")
plt.show()

In [ ]:
voyager1_distances = {"distance": interpolated_distances}
voyager1_data = pd.DataFrame(voyager1_distances)
voyager1_data = voyager1_data.assign(density = voyager1_density["vo1_dens_full"])
voyager1_data = voyager1_data.assign(vmag = voyager1_velocity["vo1_vel_full"])
voyager1_data = voyager1_data.assign(temp = voyager1_temperature["temp"])
voyager1_data = voyager1_data.assign(Bx = voyager1_mag_field["vo1_b_full[0]"])
voyager1_data = voyager1_data.assign(By = voyager1_mag_field["vo1_b_full[1]"])
voyager1_data = voyager1_data.assign(Bz = voyager1_mag_field["vo1_b_full[2]"])
voyager1_data = voyager1_data.to_csv("voyager1_data.csv")

## Voyager 2

In [ ]:
start_date = dt.datetime(2007, 3, 1, 0, 0, 0)
end_date = dt.datetime(2009, 9, 1, 0, 0, 0)

### Density against solar distance 

In [ ]:
voyager2_dist_sun = spz.amda.get_data("r_vo2", start_date, end_date)
voyager2_dist_sun = voyager2_dist_sun.to_dataframe()
voyager2_density = spz.amda.get_data("vo2_dens_full", start_date, end_date)
voyager2_density = voyager2_density.to_dataframe()
voyager2_velocity = spz.amda.get_data("vo2_vel_full", start_date, end_date)
voyager2_velocity = voyager2_velocity.to_dataframe()
voyager2_temperature = spz.amda.get_data("vo2_temp_full", start_date, end_date)
voyager2_temperature = voyager2_temperature.to_dataframe()

In [ ]:
#Manipulate the dataframes to get them into a useable form
voyager2_dist_sun.reset_index(inplace=True)
voyager2_density.reset_index(inplace=True)
voyager2_velocity.reset_index(inplace=True)
voyager2_temperature.reset_index(inplace=True)
voyager2_dist_sun = voyager2_dist_sun.rename(columns = {"index": "Time"})
voyager2_density = voyager2_density.rename(columns = {"index": "Time"})
voyager2_velocity = voyager2_velocity.rename(columns = {"index": "Time"})
voyager2_temperature = voyager2_temperature.rename(columns = {"index": "Time"})
voyager2_dist_sun["Time"] = (pd.to_datetime(voyager2_dist_sun["Time"])-start_date).dt.total_seconds()
voyager2_density["Time"] = (pd.to_datetime(voyager2_density["Time"])-start_date).dt.total_seconds()
voyager2_velocity["Time"] = (pd.to_datetime(voyager2_velocity["Time"])-start_date).dt.total_seconds()
voyager2_temperature["Time"] = (pd.to_datetime(voyager2_temperature["Time"])-start_date).dt.total_seconds()
voyager2_temperature = voyager2_temperature.assign(temp = lambda x: x["vo2_temp_full"]*1.6*10**(-23)/(1.38*10**(-23)))
print(voyager2_dist_sun.head())
print(voyager2_density.head())

In [ ]:
print(voyager2_dist_sun.tail())
print(voyager2_density.tail())
print(voyager2_temperature.tail())

In [ ]:
fig = plt.figure()
plt.plot(voyager2_density["Time"], voyager2_density["vo2_dens_full"])
plt.show()

In [ ]:
count = 14383
interpolated_times = []
while count <= 79034382:
    interpolated_times.append(count)
    count = count + 3598.381

In [ ]:
#Interpolation
interpolate_function = interpolate.interp1d(voyager2_dist_sun["Time"], voyager2_dist_sun["r_vo2"])
interpolated_distances = interpolate_function(interpolated_times)

In [ ]:
ix = [0, 1, 2, 3, 4, 5, 736, 737, 738, 739, 740, 741, 742, 743]
ixs = [730, 731, 732, 733, 734, 735, 736, 737, 738, 739, 740, 741, 742, 743]
voyager2_density = voyager2_density.drop(index=ix)
voyager2_velocity = voyager2_velocity.drop(index=ixs)
voyager2_temperature = voyager2_temperature.drop(index=ixs)
print(voyager2_density.tail())

In [ ]:
fig = plt.figure()
plt.plot(interpolated_distances, voyager2_density["vo2_dens_full"])
plt.plot(interpolated_distances, theoretical_density(interpolated_distances))
plt.legend(["Satellite Data", "Theoretical Prediction"])
plt.ylabel("Number density (cm^-3)")
plt.xlabel("Distance from the Sun (AU)")
plt.show()

In [ ]:
fig = plt.figure()
plt.plot(voyager2_dist_sun["Time"], voyager2_dist_sun["r_vo2"])
plt.show()

In [ ]:
fig = plt.figure()
plt.plot(interpolated_distances, voyager2_velocity["vo2_vel_full"])
plt.plot(interpolated_distances, theoretical_velocity(interpolated_distances))
plt.legend(["Satellite Data", "Theoretical Prediction"])
plt.ylabel("Velocity (km/s)")
plt.xlabel("Distance from the Sun (AU)")
plt.show()

In [ ]:
plt.plot(interpolated_distances, voyager2_temperature["temp"])
plt.plot(interpolated_distances, theoretical_temperature(interpolated_distances))
plt.xlabel("Distance from the Sun (AU)")
plt.ylabel("Temperature (K)")
plt.show()

In [ ]:
voyager2_distances = {"distance": interpolated_distances}
voyager2_data = pd.DataFrame(voyager2_distances)
voyager2_data = voyager2_data.assign(density = voyager2_density["vo2_dens_full"])
voyager2_data = voyager2_data.assign(vmag = voyager2_velocity["vo2_vel_full"])
voyager2_data = voyager2_data.assign(temp = voyager2_temperature["temp"])
voyager2_data = voyager2_data.to_csv("voyager2_data.csv")

In [ ]:
start_date = dt.datetime(2008, 5, 1, 0, 0, 0)
end_date = dt.datetime(2009, 6, 1, 0, 0, 0)

In [ ]:
cassini_dist_sat = spz.amda.get_data("cass_r_sat_1m", start_date, end_date)
cassini_dist_sat = cassini_dist_sat.to_dataframe()
cassini_density = spz.amda.get_data("cass_caps_n_p", start_date, end_date)
cassini_density = cassini_density.to_dataframe()

In [ ]:
cassini_dist_sat.reset_index(inplace=True)
cassini_density.reset_index(inplace=True)
cassini_dist_sat = cassini_dist_sat.rename(columns = {"index": "Time"})
cassini_density = cassini_density.rename(columns = {"index": "Time"})
cassini_dist_sat["Time"] = (pd.to_datetime(cassini_dist_sat["Time"])-start_date).dt.total_seconds()
cassini_density["Time"] = (pd.to_datetime(cassini_density["Time"])-start_date).dt.total_seconds()
print(cassini_dist_sat.head())

In [ ]:
fig = plt.figure()
plt.scatter(cassini_dist_sat["Time"], cassini_dist_sat["cass_r_sat_1m"], s=0.5)
plt.show()

In [ ]:
cassini_space_indexes = cassini_dist_sat.index[cassini_dist_sat["cass_r_sat_1m"] >= 20].to_list()
cassini_dist_space = cassini_dist_sat.filter(items = cassini_space_indexes, axis=0)
#cassini_density = cassini_density.filter(items = cassini_space_indexes, axis=0)

In [ ]:
fig = plt.figure()
plt.scatter(cassini_dist_space["Time"], cassini_dist_space["cass_r_sat_1m"], s=0.5)
plt.show()

In [ ]:
print(cassini_dist_space.tail())
print(cassini_density.tail())
fig = plt.figure()
plt.scatter(cassini_density["Time"], cassini_density["cass_caps_n_p"], s=0.5)
plt.show()

In [ ]:
fig = plt.figure()
plt.scatter(cassini_dist_space["cass_r_sat_1m"], cassini_density["cass_caps_n_p"])
plt.show()

### Creating Coordinate Frame for Jupiter

In [ ]:
from astropy.coordinates.baseframe import BaseCoordinateFrame
from astropy.coordinates import frame_transform_graph, SkyCoord
from astropy.coordinates import BaseGeodeticRepresentation, PhysicsSphericalRepresentation
from astropy import units as u
from astropy.coordinates  import solar_system_ephemeris

class JupiterSphere(BaseGeodeticRepresentation):
    equatorial_radius = 71492 * u.km
    flattening = 0.0649 * u.percent
    name = "Jupiter"
    default_representation = PhysicsSphericalRepresentation

In [ ]:
start_date = dt.datetime(1996, 1, 1, 0, 0, 0)
end_date = dt.datetime(1996, 3, 1, 0, 0, 0)

In [ ]:
galileo_dist_sun = spz.amda.get_data("gll_r_jup", start_date, end_date)
galileo_dist_sun = galileo_dist_sun.to_dataframe()

In [ ]:
galileo_dist_sun.reset_index(inplace=True)
galileo_dist_sun = galileo_dist_sun.rename(columns = {"index": "Time"})

In [ ]:
fig = plt.figure()
plt.plot(galileo_dist_sun["Time"], galileo_dist_sun["gll_r_jup"])
plt.show()

## Ulyssess

In [ ]:
#1996 solar minimum dates
start_date = dt.datetime(1996, 2, 1, 0, 0, 0)
end_date = dt.datetime(1997, 3, 1, 0, 0, 0)

In [ ]:
ulysses_1996_dist_sun = spz.get_data(amda_tree.Parameters.Ulysses.Ephemeris.ulys_orb_all.ulys_carr_r, start_date, end_date)
ulysses_1996_dist_sun = ulysses_1996_dist_sun.to_dataframe()
ulysses_1996_density = spz.amda.get_data("n_p_ulys", start_date, end_date)
ulysses_1996_density = ulysses_1996_density.to_dataframe()
ulysses_1996_velocity = spz.amda.get_data("v_ulys_rtn", start_date, end_date)
ulysses_1996_velocity = ulysses_1996_velocity.to_dataframe()
ulysses_1996_temperature = spz.amda.get_data("tp_ulys", start_date, end_date)
ulysses_1996_temperature = ulysses_1996_temperature.to_dataframe()
ulysses_1996_mag_field = spz.amda.get_data("b_ulys_rtn", start_date, end_date)
ulysses_1996_mag_field = ulysses_1996_mag_field.to_dataframe()

In [ ]:
start_date = dt.datetime(2007, 3, 1, 0, 0, 0)
end_date = dt.datetime(2008, 3, 1, 0, 0, 0)

In [ ]:
ulysses_2007_dist_sun = spz.get_data(amda_tree.Parameters.Ulysses.Ephemeris.ulys_orb_all.ulys_carr_r, start_date, end_date)
ulysses_2007_dist_sun = ulysses_2007_dist_sun.to_dataframe()
ulysses_2007_density = spz.amda.get_data("n_p_ulys", start_date, end_date)
ulysses_2007_density = ulysses_2007_density.to_dataframe()
ulysses_2007_velocity = spz.amda.get_data("v_ulys_rtn", start_date, end_date)
ulysses_2007_velocity = ulysses_2007_velocity.to_dataframe()
ulysses_2007_temperature = spz.amda.get_data("tp_ulys", start_date, end_date)
ulysses_2007_temperature = ulysses_2007_temperature.to_dataframe()
ulysses_2007_mag_field = spz.amda.get_data("b_ulys_rtn", start_date, end_date)
ulysses_2007_mag_field = ulysses_2007_mag_field.to_dataframe()

In [ ]:
start_date = dt.datetime(2008, 3, 1, 0, 0, 0)
end_date = dt.datetime(2009, 3, 1, 0, 0, 0)

In [ ]:
ulysses_2008_dist_sun = spz.get_data(amda_tree.Parameters.Ulysses.Ephemeris.ulys_orb_all.ulys_carr_r, start_date, end_date)
ulysses_2008_dist_sun = ulysses_2008_dist_sun.to_dataframe()
ulysses_2008_density = spz.amda.get_data("n_p_ulys", start_date, end_date)
ulysses_2008_density = ulysses_2008_density.to_dataframe()
ulysses_2008_velocity = spz.amda.get_data("v_ulys_rtn", start_date, end_date)
ulysses_2008_velocity = ulysses_2008_velocity.to_dataframe()
ulysses_2008_temperature = spz.amda.get_data("tp_ulys", start_date, end_date)
ulysses_2008_temperature = ulysses_2008_temperature.to_dataframe()
ulysses_2008_mag_field = spz.amda.get_data("b_ulys_rtn", start_date, end_date)
ulysses_2008_mag_field = ulysses_2008_mag_field.to_dataframe()

In [ ]:
start_date = dt.datetime(2009, 3, 1, 0, 0, 0)
end_date = dt.datetime(2009, 9, 1, 0, 0, 0)

In [ ]:
ulysses_2009_dist_sun = spz.get_data(amda_tree.Parameters.Ulysses.Ephemeris.ulys_orb_all.ulys_carr_r, start_date, end_date)
ulysses_2009_dist_sun = ulysses_2009_dist_sun.to_dataframe()
ulysses_2009_density = spz.amda.get_data("n_p_ulys", start_date, end_date)
ulysses_2009_density = ulysses_2009_density.to_dataframe()
ulysses_2009_velocity = spz.amda.get_data("v_ulys_rtn", start_date, end_date)
ulysses_2009_velocity = ulysses_2009_velocity.to_dataframe()
ulysses_2009_temperature = spz.amda.get_data("tp_ulys", start_date, end_date)
ulysses_2009_temperature = ulysses_2009_temperature.to_dataframe()
ulysses_2009_mag_field = spz.amda.get_data("b_ulys_rtn", start_date, end_date)
ulysses_2009_mag_field = ulysses_2009_mag_field.to_dataframe()

In [ ]:
ulysses_2007_9_dist_sun = pd.concat([ulysses_2007_dist_sun, ulysses_2008_dist_sun, ulysses_2009_dist_sun])
ulysses_2007_9_density = pd.concat([ulysses_2007_density, ulysses_2008_density, ulysses_2009_density])
ulysses_2007_9_velocity = pd.concat([ulysses_2007_velocity, ulysses_2008_velocity, ulysses_2009_velocity])
ulysses_2007_9_temperature = pd.concat([ulysses_2007_temperature, ulysses_2008_temperature, ulysses_2009_temperature])
ulysses_2007_9_mag_field = pd.concat([ulysses_2007_mag_field, ulysses_2008_mag_field, ulysses_2009_mag_field])

In [ ]:
ulysses_temperature.head()

In [ ]:
ulysses_dist_sun = pd.concat([ulysses_1996_dist_sun, ulysses_2007_9_dist_sun])
ulysses_density = pd.concat([ulysses_1996_density, ulysses_2007_9_density])
ulysses_velocity = pd.concat([ulysses_1996_velocity, ulysses_2007_9_velocity])
ulysses_temperature = pd.concat([ulysses_1996_temperature, ulysses_2007_9_temperature])
ulysses_mag_field = pd.concat([ulysses_1996_mag_field, ulysses_2007_9_mag_field])

In [ ]:
#Manipulate the dataframes to get them into a useable form
start_date = dt.datetime(1996, 2, 1, 0, 0, 0)
end_date = dt.datetime(1997, 3, 1, 0, 0, 0)
ulysses_dist_sun.reset_index(inplace=True)
ulysses_density.reset_index(inplace=True)
ulysses_velocity.reset_index(inplace=True)
ulysses_temperature.reset_index(inplace=True)
ulysses_mag_field.reset_index(inplace=True)
ulysses_dist_sun = ulysses_dist_sun.rename(columns = {"index": "Time"})
ulysses_density = ulysses_density.rename(columns = {"index": "Time"})
ulysses_velocity = ulysses_velocity.rename(columns = {"index": "Time"})
ulysses_temperature = ulysses_temperature.rename(columns = {"index": "Time"})
ulysses_mag_field = ulysses_mag_field.rename(columns = {"index": "Time"})
ulysses_dist_sun["Time"] = (pd.to_datetime(ulysses_dist_sun["Time"])-start_date).dt.total_seconds()
ulysses_density["Time"] = (pd.to_datetime(ulysses_density["Time"])-start_date).dt.total_seconds()
ulysses_velocity["Time"] = (pd.to_datetime(ulysses_velocity["Time"])-start_date).dt.total_seconds()
ulysses_temperature["Time"] = (pd.to_datetime(ulysses_temperature["Time"])-start_date).dt.total_seconds()
ulysses_mag_field["Time"] = (pd.to_datetime(ulysses_mag_field["Time"])-start_date).dt.total_seconds()
ulysses_velocity = ulysses_velocity.assign(vmag = lambda x: np.sqrt(x["vr"]**2 + x["vt"]**2 + x["vn"]**2))
ulysses_temperature = ulysses_temperature.assign(temp_large=lambda x: (x["t_large"]*1.6*10**(-19))/(1.38*10**(-23)))
ulysses_temperature = ulysses_temperature.assign(temp_small=lambda x: (x["t_small"]*1.6*10**(-19))/(1.38*10**(-23)))
ulysses_temperature = ulysses_temperature.assign(temp=lambda x: (x["temp_large"]+x["temp_small"])/2)

In [ ]:
print(ulysses_dist_sun.head())
print(ulysses_density.head())

In [ ]:
print(ulysses_dist_sun.tail())
print(ulysses_density.tail())
print(ulysses_velocity.tail())
print(ulysses_temperature.head())

In [ ]:
fig = plt.figure()
plt.plot(ulysses_dist_sun["Time"], ulysses_dist_sun["distance ulys-sun"])
plt.show()

In [ ]:
count = 294
interpolated_times = []
while count <= 428544000:
    interpolated_times.append(count)
    count = count + 8.2420768

In [ ]:
#Interpolation
interpolate_function_dist = interpolate.interp1d(ulysses_dist_sun["Time"], ulysses_dist_sun["distance ulys-sun"])
interpolated_distances = interpolate_function_dist(interpolated_times)
interpolate_function_den = interpolate.interp1d(ulysses_density["Time"], ulysses_density["density h+"])
interpolated_densities = interpolate_function_den(interpolated_times)
interpolate_function_vel = interpolate.interp1d(ulysses_velocity["Time"], ulysses_velocity["vmag"])
interpolated_velocities = interpolate_function_vel(interpolated_times)
interpolate_function_temp = interpolate.interp1d(ulysses_temperature["Time"], ulysses_temperature["temp"])
interpolated_temperatures = interpolate_function_temp(interpolated_times)

In [ ]:
fig = plt.figure()
plt.scatter(interpolated_distances, interpolated_densities, s=0.2)
plt.scatter(interpolated_distances, theoretical_density(interpolated_distances))
plt.ylabel("Number density (cm^-3)")
plt.xlabel("Distance from the Sun (AU)")
plt.show()

In [ ]:
fig = plt.figure()
plt.scatter(interpolated_distances, interpolated_velocities, s=0.2)
plt.scatter(interpolated_distances, theoretical_velocity(interpolated_distances))
plt.ylabel("Magnitude of the Velocity (km/s)")
plt.xlabel("Distance from the Sun (AU)")
plt.show()

In [ ]:
plt.scatter(interpolated_distances, interpolated_temperatures, s=0.2)
plt.scatter(interpolated_distances, theoretical_temperature(interpolated_distances), color="orange")
plt.xlabel("Distance from the Sun(AU)")
plt.ylabel("Temperature (K)")
plt.ylim([0, 2.5e6])
plt.show()

In [ ]:
print(len(ulysses_mag_field["Time"]))
len(interpolated_distances)

In [ ]:
plt.scatter(interpolated_distances, ulysses_mag_field["br"], s=0.2)
plt.xlabel("Distance from the Sun(AU)")
plt.ylabel("Bx (nT)")
plt.show()

In [ ]:
ulysses_distances = {"distance": interpolated_distances}
ulysses_data = pd.DataFrame(ulysses_distances)
ulysses_data = ulysses_data.assign(density = interpolated_densities)
ulysses_data = ulysses_data.assign(vmag = interpolated_velocities)
ulysses_data = ulysses_data.assign(temp = interpolated_temperatures)
ulysses_data = ulysses_data.assign(Bx = ulysses_mag_field["br"])
ulysses_data = ulysses_data.assign(By = ulysses_mag_field["bt"])
ulysses_data = ulysses_data.assign(Bz = ulysses_mag_field["bn"])
ulysses_data = ulysses_data.to_csv("ulysses_data.csv")

# Combined plot

In [ ]:
ace_data = pd.read_csv("ace_data.csv")
ace_data = ace_data.drop("Unnamed: 0", axis = 1)
print(ace_data.head())

In [ ]:
dscovr_data = pd.read_csv("dscovr_data.csv")
dscovr_data = dscovr_data.drop("Unnamed: 0", axis = 1)
print(dscovr_data.head())

In [ ]:
helios1_data = pd.read_csv("helios1_data.csv")
helios1_data = helios1_data.drop("Unnamed: 0", axis = 1)
print(helios1_data.head())

In [ ]:
psp_data = pd.read_csv("psp_data.csv")
psp_data = psp_data.drop("Unnamed: 0", axis = 1)
print(psp_data.head())

In [ ]:
voyager1_data = pd.read_csv("voyager1_data.csv")
voyager1_data = voyager1_data.drop("Unnamed: 0", axis = 1)
print(voyager1_data.head())

In [ ]:
voyager2_data = pd.read_csv("voyager2_data.csv")
voyager2_data = voyager2_data.drop("Unnamed: 0", axis = 1)
print(voyager2_data.head())

In [ ]:
ulysses_data = pd.read_csv("ulysses_data.csv")
ulysses_data = ulysses_data.drop("Unnamed: 0", axis = 1)
print(ulysses_data.head())

In [ ]:
combined_data = pd.DataFrame()
combined_data = pd.concat([ace_data, dscovr_data, helios1_data, psp_data, voyager1_data, ulysses_data]) # Removed Voyager 2
combined_data.reset_index(drop=True, inplace=True)

In [ ]:
print(combined_data.head())

In [ ]:
plt.scatter(ulysses_data["distance"], ulysses_data["vmag"], s=0.2)
#plt.scatter(combined_data["distance"], theoretical_density(combined_data["distance"]), color = "orange", s=0.5)
plt.xlabel("Distance from the Sun (AU)")
plt.ylabel("Number density (cm^-3)")
plt.show()

In [ ]:
plt.scatter(combined_data["distance"], combined_data["vmag"], s=0.5)
plt.scatter(combined_data["distance"], theoretical_velocity(combined_data["distance"]), color = "orange", s=0.5)
plt.xlabel("Distance from the Sun (AU)")
plt.ylabel("Magnitude of the velocity (km/s)")
plt.show()

In [ ]:
plt.scatter(combined_data["distance"], combined_data["temp"]/1e6, s=0.5)
plt.scatter(combined_data["distance"], theoretical_temperature(combined_data["distance"])/1e6, color = "orange", s=0.005)
plt.xlabel("Distance from the Sun (AU)")
plt.ylabel("Temperature (K)")
plt.yscale('log')
plt.show()

In [ ]:
print(combined_data.head())

# Velocity

## Solving v(r)

In [ ]:
import scipy.integrate as sci
import astropy.constants as ac
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def rc_to_au(r):
    return r*rc/AU
def cs_to_kmps(v):
    return v*cs/1e3

In [ ]:
#Constants
G = ac.G.value
Ms = ac.M_sun.value
mp = ac.m_p.value
kB = ac.k_B.value
R_Jupiter = ac.R_jup.value
R_Earth = ac.R_earth.value
R_Sun = ac.R_sun.value
AU = ac.au.value
#Parameters
T = 2e6

#Quantities
rc = G*Ms*mp/(4*kB*T); print(rc)
cs = np.sqrt(2*kB*T/mp); print(f"cs = {cs/1e3:0.0f} [km/s]")


In [ ]:
def dVdR(R, V):
    return 2*V/R * ((1-1/R)/(V**2-1))

### Solution of v(r) between 1 and 5 AU

In [ ]:
V0 = 400e3/cs
R0 = 1.00*AU / rc
RMAX = 5.00*AU / rc
Rspan = (R0, RMAX)
R_eval = np.linspace(R0, RMAX, 10000)
sol_V_R = solve_ivp(dVdR, Rspan, [V0], t_eval=R_eval)

In [ ]:
R_1AU = sol_V_R['t']
V_1AU = sol_V_R['y']

In [ ]:
plt.plot(rc_to_au(R_1AU), cs_to_kmps(V_1AU[0]))
plt.ylabel("Velocity (km/s)")
plt.xlabel("Solar Distance (AU)")

### Solution of v(r) from critical radius to 5 AU

In [ ]:
V0 = cs/cs + 0.001
R0 = rc/rc + 0.001
Rspan = (R0, (5*AU)/rc)
R_eval = np.linspace(R0, (5*AU)/rc, 10000)
sol_V_R = sci.solve_ivp(dVdR, Rspan, [V0], t_eval=R_eval)

In [ ]:
R_critical = sol_V_R['t']
V_critical = sol_V_R['y']

In [ ]:
plt.plot(rc_to_au(R_critical), cs_to_kmps(V_critical[0]))
plt.ylabel("Velocity (km/s)")
plt.xlabel("Solar Distance (AU)")

## Velocity kNN

In [ ]:
combined_dist_vel = combined_data.filter(['distance', 'vmag']).dropna()
X = combined_dist_vel["distance"].dropna()
y = combined_dist_vel["vmag"].dropna()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, test_size=0.25)

In [ ]:
X_train = [[i] for i in X_train]
X_test = [[i] for i in X_test]
y_test = [[i] for i in y_test]
y_train = [[i] for i in y_train]

In [ ]:
len(X_test)

### Sklearn Velocity kNN

In [ ]:
X_test = X_test[:-4]

In [ ]:
len(X_test)

In [ ]:
#Defining a function to apply k-nearest neighbours to the training data.
start = time.perf_counter()
knn = neighbors.KNeighborsRegressor(5000, weights="distance")
knn.fit(np.array(X_train).astype(np.float32), np.array(y_train).astype(np.float32))
split_X_train = np.array_split(X_test, 490)
velocity_pred = []
for distance in split_X_train:
    distance = [r for r_list in distance.T for r in r_list]
    distance = [[i] for i in distance]
    velocity_pred.append(knn.predict(np.array(distance).astype(np.float32)))
velocity_prediction = np.concatenate(velocity_pred)
#y_pred = knn.predict(np.array(X_test).astype(np.float32))
print(time.perf_counter()-start)

In [ ]:
velocity_prediction = [v for v_list in velocity_prediction.T for v in v_list]

In [ ]:
kNN_vel = pd.DataFrame({"distance":X_test, "velocity":velocity_prediction})
kNN_vel = kNN_vel.to_csv("kNN_vel.csv")

## Models

In [ ]:
# Köhnlein W (1996)
a1 = 2.651
a2 = 0
a3 = -0.0239
a4 = -1.836
R = np.linspace(0.1, 4.9, 10000)
def K_v(r):
    return 10**(a1+a2*np.log(r)+a3*np.exp(a4*np.log(r)))

In [ ]:
# Maruca BA et al (2023)
alpha1 = 0.157
alpha2 = 0.002
r_a = 0.72
R_initial = np.linspace(0.1, r_a, 874)
R_final = np.linspace(r_a, 4.9, 9126)
def M_v_i(r_initial):
    return 412*r_initial**alpha1
def M_v_f(r_final):
    return 412*r_a**(alpha1-alpha2)*r_final**alpha2

In [ ]:
kNN_vel = pd.read_csv("knn_vel.csv")

In [ ]:
len(kNN_vel["velocity"])

In [ ]:
#Plotting the regressed data for velocity
plt.scatter(combined_data["distance"], combined_data["vmag"], color="orange", s = 0.2)
plt.scatter(kNN_vel["distance"], kNN_vel["velocity"], color="purple", s = 0.2)
plt.scatter(rc_to_au(R_1AU), cs_to_kmps(V_1AU[0]), color="g", s=0.2)
plt.scatter(rc_to_au(R_critical), cs_to_kmps(V_critical[0]), color="olive", s=0.2)
plt.scatter(R, K_v(R), color="b", s=0.2)
plt.scatter(R_initial, M_v_i(R_initial), color="c", s=0.2)
plt.scatter(R_final, M_v_f(R_final), color="c", s=0.2)
plt.xlabel("Solar distance (AU)")
plt.ylabel("Magnitude of the velocity (km/s)")
plt.legend(["Satellite Data","kNN", "Parker model from 1 AU", "Parker model from critical radius", "Köhnlein W (1996)", "Maruca BA et al (2023)"],
           markerscale=6, handletextpad=0.05, markerfirst=False, framealpha=0.5, loc="upper right")
plt.show()

In [ ]:
#Plotting the regressed data for velocity
plt.scatter(X_test, velocity_prediction, color="purple", s = 0.2)
plt.scatter(rc_to_au(R_1AU), cs_to_kmps(V_1AU[0]), color="g", s=0.2)
plt.scatter(rc_to_au(R_critical), cs_to_kmps(V_critical[0]), color="olive", s=0.2)
plt.scatter(R, K_v(R), color="pink", s=0.2)
plt.scatter(R_initial, M_v_i(R_initial), color="c", s=0.2)
plt.scatter(R_final, M_v_f(R_final), color="c", s=0.2)
plt.xlabel("Solar distance (AU)")
plt.ylabel("Magnitude of the velocity (km/s)")
plt.legend(["Satellite Data","kNN", "Parker model from 1 AU", "Parker model from critical radius", "Köhnlein W (1996)", "Maruca BA et al (2023)"],
           markerscale=6, handletextpad=0.05, markerfirst=False, framealpha=0.5)
plt.show()

In [ ]:
knn_v = [velocity_values for velocity_values_list in velocity_prediction for velocity_values in velocity_values_list]
knn_R = [X_values for X_test_list in X_test for X_values in X_test_list]
knn_interp_func = interpolate.interp1d(knn_R, knn_v)
knn_interp = knn_interp_func(R)

In [ ]:
np.corrcoef(knn_interp, K_v(R))

In [ ]:
np.corrcoef(knn_interp, cs_to_kmps(V_critical[0]))

# Density

## Solving n(r)

In [ ]:
import scipy.integrate as sci
import astropy.constants as ac
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#Constants
AU = ac.au.value
G = ac.G.value
Ms = ac.M_sun.value
mp = ac.m_p.value
kB = ac.k_B.value
#Parameters
T = 2e6
v0 = 1.001
N0 = 1e8
nc = 5e6

#Quantities
rc = G*Ms*mp/(4*kB*T); print(f"rc = {rc/ac.R_sun.value:0.2f} R_sun")
cs = np.sqrt(2*kB*T/mp); print(f"cs = {cs/1e3:0.0f} [km/s]")

In [ ]:
R_critical = np.linspace(7e8/AU, 5, 10000)
r_1AU = np.linspace(1, 5, 10000)

In [ ]:
#Clement's suggestion:
def n1(r, V):
    return nc * (1/r)**2 / V

In [ ]:
plt.plot(R, n1(R_1AU, V_1AU[0]))
plt.plot(R, n1(R, V_critical[0]))
plt.ylabel("Number density (cm^-3)")
plt.yscale('log')
plt.xlabel("Solar Distance (AU)")

## Density kNN

In [ ]:
combined_dist_den = combined_data.filter(['distance', 'density']).dropna()
X = combined_dist_den["distance"]
y = combined_dist_den["density"]

In [ ]:
combined_dist_den.describe()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(np.array(X).astype(np.float32), np.array(y).astype(np.float32), random_state=1, test_size=0.5)

In [ ]:
X_train = [[i] for i in X_train]
X_test = [[i] for i in X_test]
y_test = [[i] for i in y_test]
y_train = [[i] for i in y_train]

In [ ]:
#Defining a function to apply k-nearest neighbours to the training data.
start = time.perf_counter()
knn = neighbors.KNeighborsRegressor(5000, weights="distance")
knn.fit(np.array(X_train).astype(np.float32), np.array(y_train).astype(np.float32))
split_X_train = np.array_split(X_test,490)
density_pred = []
for distance in split_X_train:
    distance = [r for r_list in distance.T for r in r_list]
    distance = [[i] for i in distance]
    density_pred.append(knn.predict(distance).astype(np.float32))
density_prediction = np.concatenate(density_pred)
#y_pred = knn.predict(np.array(X_test).astype(np.float32))
print(time.perf_counter()-start)

In [ ]:
density_prediction = [n for n_list in density_prediction.T for n in n_list]

In [ ]:
kNN_den = pd.DataFrame({"distance":X_test, "density":density_prediction})
kNN_den = kNN_den.to_csv("kNN_den.csv")

## Models

In [ ]:
# Köhnlein W (1996)
a1 = 0.7766
a2 = -1.934
a3 = 0.01823
a4 = -2.245

R_ = np.linspace(0.1, 4.9, 10000)
def K_n(r):
    return 10**(a1+a2*np.log(r)+a3*np.exp(a4*np.log(r)))

In [ ]:
# Maruca BA et al. (2023)
alpha1 = -2.260
alpha2 = -1.925
r_a = 2.51
R_initial = np.linspace(0.1, r_a, 4691)
R_final = np.linspace(r_a, 4.9, 5309)
def M_n_i(r_initial):
    return 5*r_initial**alpha1
def M_n_f(r_final):
    return 5*r_a**(alpha1-alpha2)*r_final**alpha2

In [ ]:
knn_den = pd.read_csv("knn_den.csv")
knn_den.head()

In [ ]:
#Plotting the regressed data for density
plt.scatter(combined_data["distance"], combined_data["density"], color="orange", s=0.2)
plt.scatter(X_test, knn_den["density"], s=0.2, color="purple")
plt.scatter(R_, n1(R_1AU, V_1AU[0]), color="g", s=0.2)
plt.scatter(R_, n1(R_, V_critical[0]), color="b", s=0.2)
#plt.scatter(R, n1(rc_to_au(R_corona), cs_to_kmps(V_corona[0])), color="b", s=0.2)
plt.scatter(R_, K_n(R_), color="pink", s=0.2)
plt.scatter(R_initial, M_n_i(R_initial), color="c", s=0.2)
plt.scatter(R_final, M_n_f(R_final), color="c", s=0.2)
plt.xlabel("Solar distance (AU)")
plt.ylabel("Number density (cm^-3)")
plt.yscale("log")
plt.legend(["Satellite Data", "kNN", "Parker model from 1 AU", "Parker model from critical radius", "Köhnlein W (1996)", "Maruca BA et al. (2023)"], 
           markerscale=6, handletextpad=0.05, markerfirst=False, framealpha=0.5)
plt.show()

In [ ]:
#Plotting the regressed data for density
plt.scatter(X_test, y_pred, s=0.2, color="purple")
plt.scatter(R, n1(rc_to_au(R_1AU), cs_to_kmps(V_1AU[0])), color="g", s=0.2)
plt.scatter(R, n1(rc_to_au(R_critical), cs_to_kmps(V_critical[0])), color="b", s=0.2)
plt.scatter(R_, K_n(R_), color="pink", s=0.2)
plt.scatter(R_initial, M_n_i(R_initial), color="c", s=0.2)
plt.scatter(R_final, M_n_f(R_final), color="c", s=0.2)
plt.xlabel("Solar distance (AU)")
plt.ylabel("Number density (cm^-3)")
plt.yscale("log")
plt.legend(["Satellite Data", "kNN", "Parker model from 1AU", "Parker model from critical radius", "Köhnlein W (1996)", "Maruca BA et al. (2023)"], 
           markerscale=6, handletextpad=0.05, markerfirst=False, framealpha=0.5)
plt.show()

In [ ]:
#knn_n = [density_values for density_values_list in density_prediction for density_values in density_values_list]
knn_R = [X_values for X_test_list in X_test for X_values in X_test_list]
knn_interp_func = interpolate.interp1d(knn_R, density_prediction)
knn_interp = knn_interp_func(R)

In [ ]:
np.corrcoef(knn_interp, M_n_i(R_initial))

In [ ]:
np.corrcoef(knn_interp, n1(R_1AU, V_1AU[0]))

# Temperature

## Solving T(r)

In [ ]:
#Constants
AU = ac.au.value
#Parameters
T0 = 2e6

In [ ]:
R = np.linspace(rc/AU, 5, 10000)
a = rc/AU

In [ ]:
def T(r):
    return T0*(a/r)**(2/3)

In [ ]:
plt.plot(R, T(R))
plt.ylabel("Temperature (K)")
plt.yscale('log')
plt.xlabel("Solar Distance (AU)")

## Temperature kNN

In [ ]:
combined_dist_temp = combined_data.filter(['distance', 'temp']).dropna()
X = combined_dist_temp["distance"]
y = combined_dist_temp["temp"]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, test_size=0.5)

In [ ]:
X_train = [[i] for i in X_train]
X_test = [[i] for i in X_test]
y_test = [[i] for i in y_test]
y_train = [[i] for i in y_train]

In [ ]:
#Defining a function to apply k-nearest neighbours to the training data.
start = time.perf_counter()
knn = neighbors.KNeighborsRegressor(5000, weights="distance")
knn.fit(np.array(X_train).astype(np.float32), np.array(y_train).astype(np.float32))
split_X_train = np.array_split(X_test, 22)
temperature_pred = []
for distance in split_X_train:
    distance = [r for r_list in distance.T for r in r_list]
    distance = [[i] for i in distance]
    temperature_pred.append(knn.predict(distance).astype(np.float32))
temperature_prediction = np.concatenate(temperature_pred)
#y_pred = knn.predict(np.array(X_test).astype(np.float32))
print(time.perf_counter()-start)

In [ ]:
temperature_prediction = [t for t_list in temperature_prediction.T for t in t_list]

In [ ]:
kNN_temp = pd.DataFrame({"distance":X_test, "temperature":temperature_prediction})
kNN_temp = kNN_temp.to_csv("kNN_temp.csv")

## Models

In [ ]:
# Köhnlein W (1996)
a1 = 4.858
a2 = -0.668
a3 = -4.90E-42
a4 = -40.91
R_ = np.linspace(0.1, 4.9, 10000)
def K_T(r):
    return 10**(a1+a2*np.log(r)+a3*np.exp(a4*np.log(r)))

In [ ]:
# Maruca BA et al. (2023)
alpha1 = -1.024
alpha2 = -0.273
r_a = 4.41
R_initial = np.linspace(0.1, r_a, 8742)
R_final = np.linspace(4.41, 4.9, 1258)
def M_T_i(r_initial):
    return T0*r_initial**alpha1
def M_T_f(r_final):
    return T0*r_a**(alpha1-alpha2)*r_final**alpha2

In [ ]:
kNN_temp = pd.read_csv("kNN_temp.csv")

In [ ]:
#Plotting the regressed data for density
plt.scatter(combined_data["distance"], combined_data["temp"], color="orange", s = 0.2)
plt.scatter(kNN_temp["distance"], kNN_temp["temperature"], color="purple", s = 0.2)
plt.scatter(R, T(R), color="g", s=0.2)
plt.scatter(R_, K_T(R_), color="pink", s=0.2)
plt.scatter(R_initial, M_T_i(R_initial), color="c", s=0.2)
plt.scatter(R_final, M_T_f(R_final), color="c", s=0.2)
plt.xlabel("Solar distance (AU)")
plt.ylabel("Temperature (K)")
plt.legend(["Satellite Data", "kNN", "Parker model", "Köhnlein W (1996)", "Maruca BA et al. (2023)"], 
           markerscale=6, handletextpad=0.05, markerfirst=False, framealpha=0.5)
plt.yscale('log')
plt.show()

In [ ]:
#Plotting the regressed data for density
plt.scatter(X_test, temperature_prediction, color="purple", s = 0.2)
plt.scatter(R, T(R), color="g", s=0.2)
plt.scatter(R_, K_T(R_), color="pink", s=0.2)
plt.scatter(R_initial, M_T_i(R_initial), color="c", s=0.2)
plt.scatter(R_final, M_T_f(R_final), color="c", s=0.2)
plt.xlabel("Solar distance (AU)")
plt.ylabel("Temperature (K)")
plt.legend(["Solar Data", "kNN", "Parker model", "Köhnlein W (1996)", "Maruca BA et al. (2023)"], 
           markerscale=6, handletextpad=0.05, markerfirst=False, framealpha=0.5)
plt.yscale('log')
plt.show()

In [ ]:
knn_T = [temperature_values for temperature_values_list in temperature_prediction for temperature_values in temperature_values_list]
knn_R = [X_values for X_test_list in X_test for X_values in X_test_list]
knn_interp_func = interpolate.interp1d(knn_R, knn_T)
knn_interp = knn_interp_func(R_initial)

In [ ]:
np.corrcoef(knn_interp, M_T_i(R_initial))

In [ ]:
np.corrcoef(knn_interp, T(R))

# Magnetic Field

## Solving B(r)

In [ ]:
b = 5e9/AU
omega = 1/(27*24*60*60)
B_theta = np.zeros(10000)
B_0 = 5
R = np.linspace(0.1, 4.9, 10000)
theta = np.linspace(0, 180, 10000)

In [ ]:
def B_r(r, theta):
    B_theta_phi = B_0*np.cos(theta)
    return B_theta_phi * (b/r)**2
def B_phi(r, theta):
    B_theta_phi = B_0*np.cos(theta)
    return B_theta_phi * (omega/cs_to_kmps(V_1AU[0])) * (r-b) * (b/r)**2 * np.sin(theta)

In [ ]:
def B_mag(r, theta):
    return (B_r(r, theta)+B_theta+B_phi(r, theta))/3

## Models

In [ ]:
# Köhnlein W (1996)
a1 = 0.500
a2 = -1.100
a3 = 0.2815
a4 = 0.875
R = np.linspace(0.1, 4.9, 10000)
def K_B(r):
    return 10**(a1+a2*np.log(r)+a3*np.exp(a4*np.log(r)))

In [ ]:
# Maruca BA et al. (2023)
alpha1 = -1.739 
alpha2 = -1.468
alpha3 = -0.904
r_a = 0.52
r_b = 4.45
R_initial = np.linspace(0.1, r_a, 875)
R_middle = np.linspace(r_a, r_b, 8188)
R_final = np.linspace(r_b, 4.9, 937)
def M_B_i(r_initial):
    return B_0*r_initial**alpha1
def M_B_m(r_middle):
    return B_0*r_a**(alpha1-alpha2)*r_middle**alpha2
def M_B_f(r_final):
    return B_0*r_a**(alpha1-alpha2)*r_b**(alpha2-alpha3)*r_final**alpha3

## Magnetic Field kNN

In [ ]:
combined_data =combined_data.assign(mag_field=lambda x: (x["Bx"]+x["By"]+x["Bz"])/3)
combined_dist_mag = combined_data.filter(['distance', 'mag_field']).dropna()
X = combined_dist_mag["distance"]
y = combined_dist_mag["mag_field"]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, test_size=0.5)

In [ ]:
X_train = [[i] for i in X_train]
X_test = [[i] for i in X_test]
y_test = [[i] for i in y_test]
y_train = [[i] for i in y_train]

In [ ]:
X_test = X_test[:-4]

In [ ]:
#Defining a function to apply k-nearest neighbours to the training data.#
start = time.perf_counter()
knn = neighbors.KNeighborsRegressor(5000, weights="distance")
knn.fit(np.array(X_train).astype(np.float32), np.array(y_train).astype(np.float32))
split_X_train = np.array_split(X_test, 490)
mag_field_pred = []
for distance in split_X_train:
    distance = [r for r_list in distance.T for r in r_list]
    distance = [[i] for i in distance]
    mag_field_pred.append(knn.predict(distance).astype(np.float32))
mag_field_prediction = np.concatenate(mag_field_pred)
#y_pred = knn.predict(np.array(X_test).astype(np.float32))
print(time.perf_counter()-start)

In [ ]:
mag_field_prediction = [B for B_list in mag_field_prediction.T for B in B_list]

In [ ]:
kNN_mag = pd.DataFrame({"distance":X_test, "mag_field":mag_field_prediction})
kNN_mag = kNN_mag.to_csv("kNN_mag.csv")

In [ ]:
kNN_mag = pd.read_csv("kNN_mag.csv")

In [ ]:
kNN_mag_field = kNN_mag["mag_field"].to_numpy()[:-4]
kNN_mag_distance = kNN_mag["distance"].to_numpy()[:-4]

In [ ]:
plt.scatter(kNN_mag["distance"], kNN_mag["mag_field"], color='purple', s=0.2)
plt.scatter(R, K_B(R), color='pink', s=0.2)
plt.scatter(R_initial, M_B_i(R_initial), color='cyan', s=0.2)
plt.scatter(R_middle, M_B_m(R_middle), color='cyan', s=0.2)
plt.scatter(R_final, M_B_f(R_final), color='cyan', s=0.2)
plt.scatter(R, B_mag(R, theta), color='green', s=0.2)
plt.yscale('log')
plt.xlabel("Solar Distance (AU)")
plt.ylabel("Magnetic Field (nT)")
plt.show()

In [ ]:
kNN_mag

In [ ]:
p_interp_func = interpolate.interp1d(kNN_mag["distance"], kNN_mag["mag_field"])
p_interp = p_interp_func(R)

In [ ]:
knn_R

In [ ]:
np.corrcoef(Com, M_B(R))

In [ ]:
np.corrcoef(kNN_mag["mag_field"], B_mag(R, theta))

In [ ]:
np.corrcoef(kNN_mag["mag_field"], B_mag(R, theta))

In [ ]:
fig, axs = plt.subplots(ncols=1, nrows=4, figsize=(5, 6), sharex=True,
                        layout="constrained")
axs[0].scatter(rc_to_au(R_1AU), cs_to_kmps(V_1AU[0]), color="g", s=0.2)
axs[0].scatter(rc_to_au(R_critical), cs_to_kmps(V_critical[0]), color="olive", s=0.2)
axs[0].set_ylabel("v (km/s)")
axs[1].scatter(R_, n1(R_1AU, V_1AU[0]), color="g", s=0.2)
axs[1].scatter(R_, n1(R_, V_critical[0]), color="b", s=0.2)
axs[1].set_yscale('log')
axs[1].set_ylabel("n (cm^-3)")
axs[2].scatter(R, T(R), color="g", s=0.2)
axs[2].set_yscale('log')
axs[2].set_ylabel("T (K)")
axs[3].scatter(R, B_mag(R, theta), color='green', s=0.2)
axs[3].set_ylabel("B (nT)")
plt.xlabel("Solar Distance (AU)")
plt.show()